In [125]:
#import packages
import csv
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import math
from joblib import dump, load

In [126]:
#return class amount
def getClassAmount(centroids):
    return len(centroids)

In [127]:
#load centroids from step2 clustering results
def getCentroids():
    cs = []
    with open("centroids.csv", "r") as csvfile:
        rows = csv.reader(csvfile)
        for row in rows:
            drow = []
            for col in row:
                drow.append(float(col))
            cs.append(drow)
    #cs = np.array(cs)
    return cs

In [128]:
#每個sample使用features如下，共11個。
#['danceability','energy','key', 'loudness', 'mode', 'speechiness', 'acousticness','instrumentalness',
# 'liveness','valence','tempo']
#classify a feature vector. x must be a 1x11 vector(11 features)
def classify(centroids,x):
    minDis = 0
    belong = 0
    for i in range(len(centroids)):
        val = 0
        for j in range(len(centroids[0])):
            val += math.pow(x[j]-centroids[i][j],2)
        if i == 0:
            minDis = val
        elif val < minDis:
            minDis = val
            belong = i
    return belong

In [129]:
#load pca model from step2
pca = load("pca.joblib")

In [130]:
#get centroids
centroids = getCentroids()

In [131]:
#get class amount
classAmount = getClassAmount(centroids)

In [132]:
#load data
data = []
city = []
header = []
if True:
    with open("city_result_2.csv", "r") as csvfile:
        rows = csv.reader(csvfile)
        i = 0
        for row in rows:
            if i == 0:
                header = row[2:13]
            else:
                data.append(row[2:13])
                city.append(row[1])
            i += 1

In [133]:
#transform data with pca model
data = np.array(data, dtype=float)
data = pca.transform(data)

In [134]:
#classify every sample in data and output nums of each class
result = []
nums = [0]*classAmount
for i in range(len(data)):
    a = [ city[i] , classify(centroids, data[i])]
    result.append(a)
    nums[classify(centroids, data[i])]+=1
print(nums)

[273, 274, 525, 437, 491]


In [135]:
# 寫入檔案
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(result)

In [136]:
# Get city - music type count
# EX ='Taipei': {0: 9, 1: 18, 2: 24, 3: 24, 4: 25}
city_music_count = {}
count = []
for r in result:
    if r[0] in city_music_count:
        if r[1] in city_music_count[r[0]]:
            city_music_count[r[0]][r[1]] += 1
    else:
        city_music_count[r[0]] = {0:0, 1:0, 2:0, 3:0, 4:0}
        city_music_count[r[0]][r[1]] += 1
print(city_music_count)

{'Taipei': {0: 9, 1: 18, 2: 24, 3: 24, 4: 25}, 'Taichung': {0: 12, 1: 14, 2: 27, 3: 24, 4: 23}, 'Tainan': {0: 14, 1: 7, 2: 20, 3: 20, 4: 39}, 'Tokyo': {0: 15, 1: 15, 2: 37, 3: 19, 4: 14}, 'Osaka': {0: 17, 1: 14, 2: 34, 3: 21, 4: 14}, 'Manila': {0: 28, 1: 12, 2: 14, 3: 30, 4: 16}, 'Kuala Lumpur': {0: 9, 1: 13, 2: 22, 3: 33, 4: 23}, 'Jakarta': {0: 18, 1: 12, 2: 27, 3: 26, 4: 17}, 'Canberra': {0: 13, 1: 12, 2: 34, 3: 18, 4: 23}, 'Wellington': {0: 17, 1: 16, 2: 28, 3: 13, 4: 26}, 'London': {0: 5, 1: 10, 2: 18, 3: 22, 4: 45}, 'Madrid': {0: 6, 1: 13, 2: 32, 3: 25, 4: 24}, 'Berlin': {0: 15, 1: 5, 2: 25, 3: 24, 4: 31}, 'Paris': {0: 9, 1: 19, 2: 41, 3: 10, 4: 21}, 'New York': {0: 12, 1: 9, 2: 24, 3: 27, 4: 28}, 'Chicago': {0: 22, 1: 16, 2: 10, 3: 31, 4: 21}, 'Ottawa': {0: 11, 1: 15, 2: 27, 3: 24, 4: 23}, 'Bras?lia': {0: 2, 1: 19, 2: 15, 3: 26, 4: 38}, 'Buenos Aires': {0: 13, 1: 24, 2: 41, 3: 7, 4: 15}, 'Berkeley California': {0: 26, 1: 11, 2: 25, 3: 13, 4: 25}}


In [137]:
# Get city - music type average
# ['Taipei', 0.09, 0.18, 0.24, 0.24, 0.25]
city_music_avg = []
city_music_avg.append(['City', '0', '1', '2', '3', '4'])
for r in city_music_count.items():
    avg_value = []
    avg_value.append(r[0])
    count = 0
    for a in r[1].items():
        count += a[1]
    avg_value.append(r[1][0]/count)
    avg_value.append(r[1][1]/count)
    avg_value.append(r[1][2]/count)
    avg_value.append(r[1][3]/count)
    avg_value.append(r[1][4]/count)
    city_music_avg.append(avg_value)

print(city_music_avg)

[['City', '0', '1', '2', '3', '4'], ['Taipei', 0.09, 0.18, 0.24, 0.24, 0.25], ['Taichung', 0.12, 0.14, 0.27, 0.24, 0.23], ['Tainan', 0.14, 0.07, 0.2, 0.2, 0.39], ['Tokyo', 0.15, 0.15, 0.37, 0.19, 0.14], ['Osaka', 0.17, 0.14, 0.34, 0.21, 0.14], ['Manila', 0.28, 0.12, 0.14, 0.3, 0.16], ['Kuala Lumpur', 0.09, 0.13, 0.22, 0.33, 0.23], ['Jakarta', 0.18, 0.12, 0.27, 0.26, 0.17], ['Canberra', 0.13, 0.12, 0.34, 0.18, 0.23], ['Wellington', 0.17, 0.16, 0.28, 0.13, 0.26], ['London', 0.05, 0.1, 0.18, 0.22, 0.45], ['Madrid', 0.06, 0.13, 0.32, 0.25, 0.24], ['Berlin', 0.15, 0.05, 0.25, 0.24, 0.31], ['Paris', 0.09, 0.19, 0.41, 0.1, 0.21], ['New York', 0.12, 0.09, 0.24, 0.27, 0.28], ['Chicago', 0.22, 0.16, 0.1, 0.31, 0.21], ['Ottawa', 0.11, 0.15, 0.27, 0.24, 0.23], ['Bras?lia', 0.02, 0.19, 0.15, 0.26, 0.38], ['Buenos Aires', 0.13, 0.24, 0.41, 0.07, 0.15], ['Berkeley California', 0.26, 0.11, 0.25, 0.13, 0.25]]


In [138]:
# 寫入檔案
# 欄位資料：['Taipei', 0.09, 0.18, 0.24, 0.24, 0.25]
with open('result.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(city_music_avg)